In [148]:
from bs4 import BeautifulSoup
import requests
import json
import urllib
import time 
import os
import pandas as pd
from glob import glob
from collections import ChainMap
from tqdm.notebook import tqdm as tqdm_notebook
from datetime import datetime 



CARRIER_CODE = ['RDWY_CRA',
                 'RDWY_FON',
                 'RDWY_FTW',
                 'RDWY_POR',
                 'RDWY_ROM',
               ]
FILE_PATH = glob(".\data\IOD_MONITORING_DETAIL_US_*.csv")[-1]
df_origin = pd.read_csv(FILE_PATH)
df = df_origin[df_origin['Carrier Code'].isin(CARRIER_CODE)]



from selenium import webdriver
import time 
browers = webdriver.Chrome("C:\\Users\\marco.wang\\Downloads\\chromedriver_win32\\chromedriver")
 
    
def check(load):
    try:
        int(load)
        return True
    except:
        return False
    
    
def clean(text):
    return "".join([ i for i in text if i.isdigit()])


result = []
loads = df["Pro No"].to_list()
for load in tqdm_notebook(loads):
    if not check(load):
        continue
    url = f'https://my.yrc.com/tools/track/shipments?referenceNumber={load}&referenceNumberType=PRO&time={int(time.mktime(datetime.now().timetuple()))}'
    browers.get(url)
    time.sleep(5)
    t = browers.find_elements_by_class_name("panel-body")
    
    d = t[1].text.split("\n")
    data = {}
    for k in range(0,len(d),2):
        data[d[k]] = d[k+1]
    data['YRC Freight PRO #:'] = clean(data['YRC Freight PRO #:'] )
    result.append({
        data['YRC Freight PRO #:']:data 
    })
    
result = dict(ChainMap(*result))
print(result)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.125)


In [ ]:
{'8588921276': {'YRC Freight PRO #:': '8588921276',
  'Trailer #:': '134194',
  'Status:': "DELIVERY DATE: 08/20/2020 TRAILER HAS BEEN SPOTTED AT A CONSIGNEE'S LOCATION. - Signed By: Drop",
  'Status Updated:': '08/20/2020',
  'Pickup Date:': '07/23/2020',
  'Delivered Date:': '08/20/2020',
  'Appointment Date:': '08/20/2020',
  'Appointment Time:': '06:00 - 07:00',
  'Ship From:': 'Ft Worth TX 76177',
  'Ship To:': 'Cromwell CT 06416'},

In [145]:

timestamp = str(datetime.now())        

today_date = timestamp[:10]
timestamp = "".join([d for d  in timestamp if d.isdigit()])
ensure_dir(today_date)
df['Status'] = df['Pro No'].apply(lambda x : result[x]['Status:'])
df['Status Updated:'] = df['Pro No'].apply(lambda x : result[x]['Status Updated:'])
df['Delivered Date'] = df['Pro No'].apply(lambda x : result[x]['Delivered Date'])
df['Appointment Date:'] = df['Pro No'].apply(lambda x : result[x]['Appointment Date:'])


df.to_excel(os.path.join(today_date,f"report_RDWY_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')

KeyError: '858892134X'

In [141]:
result

{'8588921276': {'YRC Freight PRO #:': '8588921276',
  'Trailer #:': '134194',
  'Status:': "DELIVERY DATE: 08/20/2020 TRAILER HAS BEEN SPOTTED AT A CONSIGNEE'S LOCATION. - Signed By: Drop",
  'Status Updated:': '08/20/2020',
  'Pickup Date:': '07/23/2020',
  'Delivered Date:': '08/20/2020',
  'Appointment Date:': '08/20/2020',
  'Appointment Time:': '06:00 - 07:00',
  'Ship From:': 'Ft Worth TX 76177',
  'Ship To:': 'Cromwell CT 06416'},
 '8588921743': {'YRC Freight PRO #:': '8588921743',
  'Trailer #:': '558292',
  'Status:': 'Now on Trailer 558292 Loaded for delivery',
  'Status Updated:': '08/09/2020',
  'Pickup Date:': '07/28/2020',
  'Est. Delivery Date:': '08/07/2020',
  'Ship From:': 'Ft Worth TX 76177',
  'Ship To:': 'Salt Lake City UT 84116'},
 '858892134': {'YRC Freight PRO #:': '858892134',
  'Trailer #:': '261280',
  'Status:': 'Now on Trailer 261280 Spotted at consignee',
  'Status Updated:': '08/20/2020',
  'Pickup Date:': '07/27/2020',
  'Est. Delivery Date:': '07/31/202